In [1]:
import pandas as pd
from gensim import models
import numpy as np
import gensim
import gensim.downloader
from tqdm import tqdm
import time
import os

In [2]:
dictionary = pd.read_csv('data/stanfordSentimentTreebank/dictionary.txt', header=None, sep='|')
dictionary = dictionary.rename(columns={0: 'sentence', 1: 'phrase ids'})

dictionary

,sentence,phrase ids
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936
...,...,...
239227,zoning ordinances to protect your community fr...,220441
239228,zzzzzzzzz,179256
239229,élan,220442
239230,É,220443


In [3]:
sentiment_labels = pd.read_csv('data/stanfordSentimentTreebank/sentiment_labels.txt', header=0, sep='|')
sentiment_labels

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708
...,...,...
239227,239227,0.36111
239228,239228,0.38889
239229,239229,0.33333
239230,239230,0.88889


In [4]:
def transform_df(df):
    df = pd.merge(df, dictionary, on='sentence')
    df = pd.merge(df, sentiment_labels, on='phrase ids')
    df['label'] = 1
    df.loc[df['sentiment values'] <= 0.4, 'label'] = 0
    df = df.drop(df[ (df['sentiment values'] > 0.4) & (df['sentiment values'] <= 0.6)].index).reset_index(drop=True)
    
    df = df[['label', 'sentence']]
    return df

In [5]:
df = pd.read_csv('data/sentiment_treebank/data.csv')
df_train = df[df.splitset_label == 1].reset_index(drop=True)
df_eval = df[df.splitset_label == 3].reset_index(drop=True)
df_test = df[df.splitset_label == 2].reset_index(drop=True)

In [6]:
df_train = transform_df(df_train)
df_eval = transform_df(df_eval)
df_test = transform_df(df_test)

In [7]:
df_train

,label,sentence
0,0,Suffers from a flat script and a low budget .
1,0,It is n't scary .
2,1,"It 's a quirky , off-beat project ."
3,0,Cox offers plenty of glimpses at existing phot...
4,0,Too smart to ignore but a little too smugly su...
...,...,...
6563,0,An almost unbearably morbid love story .
6564,1,This film puts Wang at the forefront of China ...
6565,0,"This is an ungainly movie , ill-fitting , with..."
6566,0,Although the sequel has all the outward elemen...


In [8]:
df_train.to_csv('data/sentiment_treebank/train.csv', index=False)
df_eval.to_csv('data/sentiment_treebank/eval.csv', index=False)
df_test.to_csv('data/sentiment_treebank/test.csv', index=False)